# Monte Carlo Approximation von Pi

## Python-Implementierung

In [1]:
import time
import timeit
import random

In [2]:
def mc_pi(N: int = 100000):
    in_ = 0

    for i in range(N):
        x = random.random()
        y = random.random()

        if x*x + y*y < 1:
            in_ += 1
    
    return 4*in_ / N

In [3]:
N = 10000000
t1 = time.perf_counter()
pi = mc_pi(N)
t2 = time.perf_counter()
dt = t2-t1
print('Pi mit %d iterationen: %.7f, Zeit %.7f s' % (N, pi, dt))

Pi mit 10000000 iterationen: 3.1419164, Zeit 7.7041630 s


In [4]:
%%timeit -n 10
pi = mc_pi(1000000)

723 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%prun
pi = mc_pi(1000000)

In [6]:
%load_ext Cython

In [7]:
%%cython -a

import random

def mc_pi_cython1(long N = 100000):
    cdef long in_ = 0
    cdef long i

    cdef double x, y

    for i in range(N):
        x = random.random()
        y = random.random()

        if x*x + y*y < 1:
            in_ += 1
    
    return (4.0*in_) / N

In [8]:
%%timeit -n 10
pi = mc_pi_cython1(1000000)

192 ms ± 5.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%cython -a

import numpy as np
cimport numpy as np

def mc_pi_cython2(long N = 100000):
    cdef long in_ = 0
    cdef long i

    cdef double x, y

    for i in range(N):
        x = np.random.random()
        y = np.random.random()

        if x*x + y*y < 1:
            in_ += 1
    
    return (4.0*in_) / N

In [16]:
%%timeit -n 10
pi = mc_pi_cython2(1000000)

910 ms ± 47.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%cython -a

from libc.stdlib cimport rand, RAND_MAX

def mc_pi_cython3(long N = 100000):
    cdef long in_ = 0
    cdef long i

    cdef double x, y

    for i in range(N):
        x = rand() / RAND_MAX
        y = rand() / RAND_MAX

        if x*x + y*y < 1:
            in_ += 1
    
    return (4.0*in_) / N

In [19]:
%%timeit -n 10
pi = mc_pi_cython3(1000000)
print(pi)

3.139088
3.139416
3.141016
3.142868
3.143576
3.140688
3.142228
3.142256
3.141376
3.136812
3.140372
3.141708
3.141572
3.139304
3.140544
3.142796
3.14008
3.142776
3.14258
3.141156
3.14098
3.141576
3.139568
3.141528
3.142944
3.144136
3.144764
3.141256
3.141732
3.138964
3.144532
3.141584
3.139396
3.143636
3.139868
3.141888
3.139488
3.141344
3.141856
3.141292
3.1395
3.14228
3.14152
3.141264
3.141652
3.144356
3.143892
3.141428
3.140084
3.137292
3.138432
3.142032
3.140352
3.142348
3.14116
3.14396
3.143336
3.140412
3.13862
3.138484
3.1405
3.141532
3.145668
3.141028
3.139904
3.14232
3.141416
3.141316
3.138568
3.141732
65.4 ms ± 4.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
import threading

num = 0
lock = threading.Lock()

def mc_pi_thread(N: int = 100000):
    global num, lock
    in_ = 0

    for i in range(N):
        x = random.random()
        y = random.random()

        if x*x + y*y < 1:
            in_ += 1
    
    with lock:
        num += in_

Nt = 8

In [56]:
t1 = time.perf_counter()
tlist = []
N = 1000000
for it in range(Nt):
    t = threading.Thread(target=mc_pi_thread, args=(N//Nt,))
    t.start()
    tlist.append(t)

for thread in tlist:
    thread.join()
t2 = time.perf_counter()
print('pi: %.7f (%.6f s)' % (num/N*4.0, (t2-t1)/Nt))

pi: 3.1426160 (0.126467 s)
